<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U11/ANN11_LSTM_Shakespeare_SOLUTION_pl.ipynb)

# Dichten wie Shakespeare ...

## Rekurrente Netze - LSTMs

- LSTMs führen das Konzept eines Zustands für jede Schicht in einem rekurrenten Netz ein. 
- Der Zustand fungiert als sein Speicher. Man kann sich das wie das Hinzufügen von Attributen zu einer Klasse in der objektorientierten Programmierung vorstellen. 
- Die Attribute des Speicherzustands werden mit jedem Trainingsbeispiel aktualisiert.

- In LSTMs sind die Regeln, welche die im State (Speicher) gespeicherten Informationen bestimmen, trainierte neuronale Netze - darin liegt der Magie (die Intelligenz?). Sie können trainiert werden, um zu lernen, was sie sich merken sollen, während gleichzeitig der Rest des rekurrenten Netzes lernt, das Ziel-Label vorherzusagen! 
- Mit der Einführung eines Speichers (memory) und eines Zustands (state) ist das Modell in der Lage, Abhängigkeiten zu lernen, die sich nicht nur über ein oder zwei Token, sondern über die die Gesamtheit jeder Datenprobe erstrecken. 

Mit diesen langfristigen Abhängigkeiten in der Hand, kann man über die Wörter selbst hinausgehen und etwas Tieferes über die Sprache herausfinden. Mit LSTMs stehen dem Modell Muster zur Verfügung, die der Mensch als selbstverständlich ansieht und auf einer unterbewussten Ebene verarbeitet. Und mit diesen Mustern können Sie nicht nur
Muster genauer vorhersagen, sondern auch **neue Texte generieren**. Der Stand der Technik in diesem Bereich ist noch lange nicht perfekt, aber die aber die Ergebnisse, die Sie sehen werden, selbst in Ihren Spielzeugbeispielen, sind beeindruckend.



## Modellierung von Sprache auf Zeichenebene

Worte haben eine Bedeutung - da sind wir uns alle einig. Die Modellierung natürlicher Sprache mit diesen
Grundbausteinen zu modellieren, erscheint daher nur natürlich (WordVectors). Die Verwendung dieser Modelle zur Beschreibung von Bedeutung, Gefühle, Absichten und alles andere in Form dieser atomaren Strukturen zu beschreiben, scheint
ebenfalls natürlich. 

Aber natürlich sind Wörter überhaupt nicht atomar. Wie Sie vorhin gesehen haben, bestehen sie aus kleineren Wörtern, Wortstämmen, Phonemen und so weiter. Aber sie sind auch, und das ist noch grundlegender, eine **Folge von Zeichen**.


- Bei der *Modellierung von Sprache* ist ein Grossteil der Bedeutung *auf der Zeichenebene* verborgen.
- *Intonationen* in der Stimme, *Alliterationen*, *Reime* - all das kann modelliert werden, wenn man wenn man die Dinge bis auf die Zeichenebene herunterbricht. 

Viele dieser Muster sind in einem Text enthalten, wenn man den Text daraufhin untersucht, welches Zeichen $x_k$ nach welchem folgt, unter Berücksichtigung (conditional probability) der Zeichen, die vorangegangen sind.

$$ p(x_k \vert x_{k-1}, x_{k-1}, \dots , x_{k-m})$$


In diesem Paradigma wird ein Leerzeichen, ein Komma oder ein Punkt einfach zu einem weiteren Zeichen.
Und da das LSTM-Netz die **Bedeutung von Sequenzen lernt**, ist es gezwungen, Muster auf niedrigerer Ebene zu finden.
Wenn das Modell nach einer bestimmten Anzahl von Silben ein sich wiederholendes Suffix erkennt, das sich
reimt, könnte dies ein Hinweis auf eine Bedeutung wie vielleicht Heiterkeit oder Spott sein.

- Mit einer ausreichend grossen Trainingsmenge beginnen sich diese Muster herauszukristallisieren. Und da es in der englischen Sprache viel weniger unterschiedliche Buchstaben als Wörter gibt, muss sich das Modell um eine eine geringere Vielfalt an Eingabevektoren kümmern (*Dimensionsreduktion* des Inputs).
- **Das Trainieren eines Modells auf Buchstabenebene ist jedoch schwierig**. Die Muster und langfristigen Abhängigkeiten, die auf der Zeichenebene zu finden sind, können von Text zu Text sehr unterschiedlich sein. Sie können diese Muster finden, aber sie lassen sich möglicherweise nicht so gut verallgemeinern. 


# Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn

# Check if CUDA (GPU) is available, otherwise use CPU
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# (a) Herunterladen und Speichern des Datensatzes 

1. **Herunterladen des Textes:**
   - Es wird eine Verbindung zu einer URL aufgebaut, wo sich ein Text mit Werken von Shakespeare befindet.
   - Der komplette Text wird aus dem Internet geladen und im Arbeitsspeicher gespeichert.

2. **Speichern auf dem Computer:**
   - Der geladene Text wird in eine neue Datei namens `shakespeare.txt` geschrieben.
   - Diese Datei liegt dann lokal auf deinem Rechner.

3. **Rückmeldung:**
   - Nach dem erfolgreichen Speichern gibt das Skript eine einfache Bestätigung aus:
     ```
     Text file downloaded and saved.
     ```

**Ziel dieses Skripts:**
Der heruntergeladene Text kann später verwendet werden – z. B. zum Trainieren eines LSTM-Modells für Textgenerierung (wie bei KI-Schreibassistenten oder automatischem Dichten).

In [ ]:
import requests

# URL of the text file
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/678ae4873e82f52ae1b563e32c12c6837fd5ae78/data/Gutenberg/shakespeare.txt"

# Download the file
response = requests.get(url)
shakespeare_text = response.text

# Save the text to a local file (optional)
with open("shakespeare.txt", "w", encoding="utf-8") as file:
    file.write(shakespeare_text)

print("Text file downloaded and saved.")


# (b) Datensatz lesen
In diesem Code wird die Datei `shakespeare.txt` geöffnet und ihr gesamter Inhalt wird eingelesen. Das passiert in einem sogenannten `with`-Block.

### Schritt-für-Schritt:

1. **Datei öffnen:**
   Die Textdatei `shakespeare.txt` wird geöffnet. Der `with`-Block sorgt dafür, dass die Datei nach dem Lesen automatisch wieder geschlossen wird – auch wenn ein Fehler auftritt.

2. **Inhalt lesen:**
   Mit `f.read()` wird der gesamte Textinhalt der Datei eingelesen und in der Variable `text` gespeichert. Der Inhalt liegt danach als ein langer String vor.

## Ergebnis

Der gesamte Text aus der Datei befindet sich nun in der Variable `text` und kann weiterverarbeitet werden, z. B. zur Tokenisierung oder für ein LSTM-Sprachmodell.


In [ ]:
with open("shakespeare.txt") as f:
    text = f.read()


# (c) Zeichenbasierte Kodierung des Textes

Dieser Code bereitet den Shakespeare-Text für die zeichenweise Verarbeitung vor (z. B. für ein LSTM-Modell).

## Schritt-für-Schritt-Erklärung

1. **`all_char = set(text)`**  
   → Erstellt eine Menge aller **einzigartigen Zeichen** im Text (z. B. Buchstaben, Satzzeichen, Leerzeichen).

2. **`n_unique_char = len(all_char)`**  
   → Zählt, wie viele **unterschiedliche Zeichen** es gibt (z. B. 65 oder 80).

3. **`decoder = dict(enumerate(all_char))`**  
   → Erstellt ein Wörterbuch, das jedem Index ein Zeichen zuordnet.  
   → Wird später verwendet, um Zahlen wieder in Text umzuwandeln (**Decodierung**).

4. **`encoder = {char: ind for ind, char in decoder.items()}`**  
   → Erstellt das Gegenstück zum Decoder: ein Wörterbuch, das jedem Zeichen einen Index zuordnet (**Encodierung**).

5. **`encoded_text = np.array([encoder[char] for char in text])`**  
   → Wandelt den gesamten Text in eine Folge von Ganzzahlen um.  
   → Jedes Zeichen wird durch seine entsprechende Zahl aus dem `encoder` ersetzt.

## Ergebnis

Der Text ist jetzt numerisch kodiert – als NumPy-Array mit Ganzzahlen – und bereit für das Training eines Modells wie LSTM oder GRU.

In [ ]:
all_char = set(text)
n_unique_char = len(all_char)
decoder = dict(enumerate(all_char))
encoder = {char: ind for ind, char in decoder.items()}
encoded_text = np.array([encoder[char] for char in text])


# (d) One-Hot-Encoding-Funktion

## Funktion: `one_hot_encoder(encoded_text, n_unique_char)`

Diese Funktion wandelt eine Folge von ganzzahligen Zeichen-IDs in ein **One-Hot-Encoded Format** um.

### Was passiert?

- Erstellt eine Null-Matrix der Form `(Anzahl_Zeichen, Anzahl_einzigartiger_Zeichen)`.
- Setzt für jedes Zeichen an der passenden Stelle den Wert auf `1.0`.
- Formt das Ergebnis zurück zur Originalform, ergänzt um die One-Hot-Dimension.

### Eingabe:
- `encoded_text`: 1D- oder 2D-Array mit ganzzahligen Zeichen-IDs.
- `n_unique_char`: Anzahl aller unterschiedlichen Zeichen.

### Ausgabe:
- Ein 3D-Array mit One-Hot-Vektoren (z. B. `[10000, 65] → [10000, 65]` oder `[B, T] → [B, T, C]`).
 Wird häufig für zeichenbasiertes Training von RNNs oder LSTMs genutzt.

In [ ]:
def one_hot_encoder(encoded_text, n_unique_char):
    one_hot = np.zeros((encoded_text.size, n_unique_char)).astype(np.float32)
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
    one_hot = one_hot.reshape((*encoded_text.shape, n_unique_char))
    return one_hot


# (e) Funktion: `generate_batches`

Diese Funktion erstellt **Mini-Batches** für das Training eines RNN-/LSTM-Modells auf Zeichenebene.

## Zweck
Sie teilt den kodierten Text in Trainingsbeispiele mit fester Sequenzlänge auf – inklusive der passenden Zielsequenz (shifted by one character).

## Parameter
- `encoded_text`: Der numerisch kodierte Text (als NumPy-Array).
- `sample_per_batch`: Wie viele Sequenzen pro Batch erzeugt werden.
- `seq_len`: Länge jeder Sequenz (in Zeichen).

## Ausgabe (via `yield`)
- `x`: Eingabesequenzen (Größe: `[batch_size, seq_len]`)
- `y`: Zielsequenzen (jeweils um 1 Zeichen nach rechts verschoben)

## Besonderheiten
- Wenn am Ende nicht genug Zeichen für eine vollständige Sequenz vorhanden sind, wird zyklisch der Anfang verwendet.
- Die Funktion ist ein Generator – ideal für die sequentielle Datenverarbeitung im Training.

Perfekt für zeichenbasierte Textgenerierung mit LSTMs.

In [ ]:
def generate_batches(encoded_text, sample_per_batch=10, seq_len=50):
    char_per_batch = sample_per_batch * seq_len
    avail_batch = int(len(encoded_text) / char_per_batch)
    encoded_text = encoded_text[: char_per_batch * avail_batch]
    encoded_text = encoded_text.reshape((sample_per_batch, -1))

    for n in range(0, encoded_text.shape[1], seq_len):
        x = encoded_text[:, n : n + seq_len]
        y = np.zeros_like(x)
        try:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, n + seq_len]
        # for the very last case
        except:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, 0]
        yield x, y


# (f) Klasse: `LSTM` (zeichenbasiertes Textgenerierungsmodell)

Dieses LSTM-Modell ist für zeichenweise Textgenerierung konzipiert (z. B. mit Shakespeare-Texten).

## Initialisierung (`__init__`)
- **`all_char`**: Liste aller einzigartigen Zeichen im Text.
- **`num_hidden`**: Anzahl der Neuronen im versteckten Zustand.
- **`num_layers`**: Anzahl der gestapelten LSTM-Schichten.
- **`drop_prob`**: Dropout-Rate zur Regularisierung.

### Bestandteile:
- **`encoder / decoder`**: Mapping zwischen Zeichen und Indizes.
- **`self.lstm`**: Mehrschichtiges LSTM mit `batch_first=True`.
- **`self.fc_linear`**: Lineare Schicht zur Projektion auf den Zeichenvorrat.
- **`self.dropout`**: Dropout-Schicht zur Vermeidung von Overfitting.

## Vorwärtsdurchlauf (`forward`)
- Führt den Input durch das LSTM und Dropout.
- Formt die Ausgabe in 2D um und leitet sie durch die lineare Schicht.
- Gibt Vorhersagen (`final_out`) und den neuen Hidden State zurück.

## Hidden-State-Initialisierung (`init_hidden`)
- Erstellt initiale Hidden States (Hidden + Cell) mit Nullen.
- Achtet auf Gerätekompatibilität (`to(device)`).

Dieses Modell kann verwendet werden, um Zeichenfolgen zu generieren, basierend auf trainierten Wahrscheinlichkeiten für Zeichenübergänge.

In [ ]:
class LSTM(nn.Module):
    def __init__(self, all_char, num_hidden=512, num_layers=3, drop_prob=0.5):
        super(LSTM, self).__init__()
        self.all_char = all_char
        self.num_hidden = num_hidden
        self.num_layers = num_layers
        self.drop_prob = drop_prob

        self.decoder = dict(enumerate(all_char))
        self.encoder = {char: ind for ind, char in decoder.items()}

        self.lstm = nn.LSTM(
            len(self.all_char),
            num_hidden,
            num_layers,
            dropout=drop_prob,
            batch_first=True,
        )
        self.fc_linear = nn.Linear(num_hidden, len(self.all_char))
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x, hidden):
        lstm_out, hidden = self.lstm(x, hidden)
        drop_out = self.dropout(lstm_out)
        drop_out = drop_out.contiguous().view(-1, self.num_hidden)
        final_out = self.fc_linear(drop_out)
        return final_out, hidden

    def init_hidden(self, batch_size):
        hidden = (
            torch.zeros(self.num_layers, batch_size, self.num_hidden).to(device),
            torch.zeros(self.num_layers, batch_size, self.num_hidden).to(device),
        )
        return hidden


Hier wird das Modell trainiert.

In [ ]:
model = LSTM(all_char, num_hidden=1024, num_layers=4, drop_prob=0.6).to(device)


# (g) Trainingseinstellungen und Datenaufteilung

Dieser Code bereitet alles für das Training eines LSTM-Modells zur Zeichen-Textgenerierung vor.

## Optimierung & Verlustfunktion
- **`optimizer`**: Verwendet den Adam-Optimizer mit Lernrate 0.001.
- **`criterion`**: Nutzt `CrossEntropyLoss`, passend für Klassifikation über Zeichen.

## Trainings-/Validierungsaufteilung
- **`train_percent = 0.9`**: 90 % der Daten werden für das Training verwendet.
- **`train_data`**: Enthält den vorderen Teil des kodierten Textes (Training).
- **`val_data`**: Der hintere Teil wird für die Validierung genutzt.

## Trainingsparameter
- **`num_epoch = 10`**: Das Modell wird über 10 Epochen trainiert.
- **`batch_size = 100`**: Jeweils 100 Zeichen-Sequenzen pro Batch.
- **`seq_len = 100`**: Jede Eingabesequenz hat eine Länge von 100 Zeichen.
- **`tracker = 0`**: Zähler (vermutlich für späteres Tracking wie Verlust).
- **`num_char`**: Anzahl der einzigartigen Zeichen (für Output-Größe).

## Modell in Trainingsmodus
- **`model.train()`**: Schaltet das Modell in den Trainingsmodus (z. B. für Dropout aktiv).

Damit ist das Modell bereit für den Trainingsloop.

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
train_percent = 0.9
train_ind = int(len(encoded_text) * (train_percent))
train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]
num_epoch = 10
batch_size = 100
seq_len = 100
tracker = 0
num_char = max(encoded_text) + 1
# set the model on training mode
model.train()


# (h) LSTM-Trainingsloop zur Zeichen-Textgenerierung

Dieser Code führt das Training des LSTM-Modells über mehrere Epochen durch und validiert regelmäßig den Fortschritt.

## Vorbereitung
- **`torch.cuda.empty_cache()`**: Gibt nicht verwendeten GPU-Speicher frei.
- **`model.train()`**: Aktiviert den Trainingsmodus (z. B. für Dropout).
- **Batch-Größe & Sequenzlänge**: 100 Sequenzen à 100 Zeichen pro Batch.

---

## Epochenschleife (`for epoch in range(num_epoch)`)
### Pro Epoche:
1. **Hidden-State zurücksetzen**  
   Initialisiert den versteckten Zustand für das LSTM.

2. **Durch Batch-Generator iterieren**  
   Holt Eingabe- und Zielsequenzen (`x`, `y`) aus `generate_batches`.

3. **Vorverarbeitung und Training**
   - Leert GPU-Speicher erneut.
   - Setzt Gradienten auf 0.
   - One-Hot-Encodiert die Eingaben.
   - Wandelt Eingaben und Ziele in Torch-Tensoren und überträgt sie aufs Gerät.
   - Führt einen Vorwärtsdurchlauf durchs Modell.
   - Berechnet den Verlust (`CrossEntropyLoss`).
   - Führt Backpropagation durch.
   - Clipped Gradienten zur Stabilisierung.
   - Optimierungsschritt mit Adam.

---

## Validierung (alle 25 Schritte)
- Modell wird auf `eval()` gesetzt (kein Dropout).
- Führt Vorhersagen auf Trainingsdaten aus.
- Berechnet und speichert Validierungsverluste.
- Gibt aktuelle Metriken aus:
  ```text
  epoch : <nr> tracker : <nr> loss : <wert>

In [ ]:
torch.cuda.empty_cache()  # Clear unused GPU memory
model.train()

# Reduce batch size and sequence length to lower memory usage
batch_size = 100  # Reduced from 100
seq_len = 100  # Reduced from 100

# For every epoch
for epoch in range(num_epoch):
    # Reset hidden state
    hidden = model.init_hidden(batch_size)
    # Go through every x, y in batch_gen obj
    for x, y in generate_batches(val_data, batch_size, seq_len):
        tracker += 1
        # Clear unused GPU memory
        torch.cuda.empty_cache()
        # Zero_grad
        model.zero_grad()
        # Create input and target
        x = one_hot_encoder(x, num_char)
        inputs = torch.tensor(x).to(device)
        targets = torch.tensor(y).long().to(device)
        hidden = tuple([state.data for state in hidden])
        # Now pass through model
        lstm_out, hidden = model.forward(inputs, hidden)
        # Calculate loss and backprop
        loss = criterion(lstm_out, targets.view(batch_size * seq_len))
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()
        # For every 25 steps do validation
        if tracker % 25 == 0:
            # Put model in eval mode
            model.eval()
            val_hidden = model.init_hidden(batch_size)
            val_losses = []
            for x, y in generate_batches(train_data, batch_size, seq_len):
                x = one_hot_encoder(x, num_char)
                inputs = torch.tensor(x).to(device)
                targets = torch.tensor(y).long().to(device)
                val_hidden = tuple([state.data for state in val_hidden])
                lstm_out, val_hidden = model.forward(inputs, val_hidden)
                loss = criterion(lstm_out, targets.view(batch_size * seq_len))
                val_losses.append(loss.item())
            print(f"epoch : {epoch + 1} tracker : {tracker} loss : {loss.item()}")
            model.train()


# (i) Funktion: `pred_next_char`

Diese Funktion sagt das **nächste Zeichen** vorher, basierend auf einem gegebenen Zeichen und dem aktuellen Zustand des LSTM-Modells.

## Zweck
Sie dient zur **zeichengenauen Textgenerierung**, z. B. um ein neues Shakespeare-ähnliches Gedicht zu erzeugen.

## Schritt-für-Schritt

1. **Eingabe-Zeichen kodieren:**
   - Das Zeichen wird in einen Integer-Index umgewandelt (`encoder`).
   - Danach wird es in ein One-Hot-Vektor umgewandelt.

2. **Tensor erstellen:**
   - Der One-Hot-Vektor wird in einen Torch-Tensor umgewandelt und aufs richtige Gerät (GPU/CPU) verschoben.

3. **Hidden-State vorbereiten:**
   - Der versteckte Zustand wird aktualisiert und auf `.data` gesetzt (für detach von Autograd).

4. **Vorhersage machen:**
   - Der Tensor wird durch das LSTM-Modell geschickt.
   - Die Ausgabe wird mit `softmax` in Wahrscheinlichkeiten umgewandelt.

5. **Top-k Sampling:**
   - Es werden die `k` wahrscheinlichsten Zeichen gewählt.
   - Eins davon wird zufällig (aber gewichtet durch die Wahrscheinlichkeiten) ausgewählt.

6. **Ausgabe:**
   - Gibt das vorhergesagte Zeichen (als Symbol) zurück sowie den aktualisierten Hidden-State.

## Parameter
- `model`: Das trainierte LSTM-Modell.
- `char`: Das aktuelle Zeichen.
- `hidden`: Der aktuelle versteckte Zustand des Modells.
- `k`: Top-k-Sampling – wie viele mögliche Zeichen zur Auswahl stehen.

**Anwendung:** Diese Funktion wird wiederholt aufgerufen, um Zeichen für Zeichen neuen Text zu generieren.

In [ ]:
def pred_next_char(model, char, hidden=None, k=1):
    encoded_text = model.encoder[char]
    encoded_text = np.array([[encoded_text]])
    encoded_text = one_hot_encoder(encoded_text, len(model.all_char))
    # create input by encoding and one_hotting the char
    inputs = torch.tensor(encoded_text).to(device)
    # create hidden state
    hidden = tuple([state.data for state in hidden])
    # make prediction
    lstm_out, hidden = model.forward(inputs, hidden)
    # get probabilities
    probs = torch.nn.functional.softmax(lstm_out, dim=1).data
    probs = probs.cpu()
    probs, index_position = probs.topk(k)
    index_position = index_position.numpy().squeeze()
    probs = probs.numpy().flatten()
    probs = probs / probs.sum()
    # choose a char from top k
    char = np.random.choice(index_position, p=probs)
    return model.decoder[char], hidden


# (k) Funktion: `generate_text`

Diese Funktion **generiert neuen Text** Zeichen für Zeichen mithilfe eines trainierten LSTM-Modells.

## Schritt-für-Schritt-Erklärung

1. **Modell vorbereiten:**
   - Das Modell wird auf das richtige Gerät (`CPU` oder `GPU`) verschoben.
   - Es wird in den Evaluierungsmodus (`eval()`) versetzt, um Dropout & Co. zu deaktivieren.

2. **Initialisierung:**
   - Das Startwort (`seed`) wird in eine Liste von Zeichen (`output_char`) zerlegt.
   - Der Hidden-State des LSTM wird für Batch-Größe 1 initialisiert.

3. **Seed "verarbeiten":**
   - Für jedes Zeichen im Seed wird `pred_next_char` aufgerufen, um den Hidden-State schrittweise aufzubauen.
   - Das letzte vorhergesagte Zeichen wird zur Ausgabe hinzugefügt.

4. **Textgenerierung:**
   - Es werden `size` weitere Zeichen generiert.
   - In jedem Schritt wird das zuletzt erzeugte Zeichen als Eingabe verwendet.
   - Das neue Zeichen wird an die Ausgabeliste angehängt.

5. **Rückgabe:**
   - Die generierten Zeichen werden durch Leerzeichen getrennt und als String zurückgegeben.

## Parameter

- `model`: Das trainierte LSTM-Modell.
- `size`: Anzahl der zu generierenden Zeichen.
- `seed`: Startzeichen oder -wort für den Text.
- `k`: Wie viele Zeichen für Top-k-Sampling berücksichtigt werden.

**Ziel:** Automatische Texterzeugung, z. B. in Shakespeare-Stil oder basierend auf einem anderen Textkorpus.

In [ ]:
def generate_text(model, size, seed="the", k=1):
    model = model.to(device)
    model.eval()
    output_char = [c for c in seed]
    hidden = model.init_hidden(1)
    for char in seed:
        char, hidden = pred_next_char(model, char, hidden, k=k)
    output_char.append(char)
    for i in range(size):
        char, hidden = pred_next_char(model, output_char[-1], hidden, k=k)
        output_char.append(char)
    return " ".join(output_char)


# (l) Vorhersage 

## Was passiert hier?

Dieser Befehl ruft die Funktion `generate_text(...)` auf und gibt das Ergebnis direkt in der Konsole aus.

## Bedeutung der Parameter:

- `model`: Das trainierte LSTM-Modell zur Textgenerierung.
- `200`: Anzahl der zu generierenden Zeichen.
- `"The "`: Start-Seed, mit dem der Text beginnt.
- `k=2`: **Top-2-Sampling** – bei jeder Vorhersage werden die 2 wahrscheinlichsten nächsten Zeichen berücksichtigt, eines davon wird zufällig gewählt (mit gewichteter Wahrscheinlichkeit).

## Ausgabe:

Das Ergebnis ist ein automatisch generierter Text, der mit `"The "` beginnt und dann 200 Zeichen lang fortgesetzt wird – im Stil des Trainingskorpus (z. B. Shakespeare).  
Der generierte Text sieht z. B. so aus:

In [ ]:
print(generate_text(model, 200, seed="The ", k=2))


https://medium.com/@adi.joshi2018/building-a-character-level-language-model-from-scratch-with-pytorch-9fe248525f1c
